In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import shutil
import pandas as pd
import ray
from toolkit_run.ray.server import LabRayToolkitServer
import json

In [ ]:
server = LabRayToolkitServer()
server.dashboard_url

In [ ]:
server.scale_cluster(60)

## Repos count and size of files for java

In [ ]:
paths = list(Path('/data/hf_repos/the-stack-metadata').glob('data/*'))

In [ ]:
@ray.remote(scheduling_strategy="SPREAD")
def get_java_repos_stats(bucket_path):
    df_fi = pd.read_parquet(
        bucket_path / 'fi.parquet'
    )
    df_fi = df_fi[(df_fi['lang_ex'] == 'Java') & (df_fi['is_deleted'] == False)]
    return df_fi['ri_id'].nunique(), df_fi['size'].sum()
    
    

In [ ]:
res = []
for path in paths:
    res += [get_java_repos_stats.remote(path)]


In [ ]:
res = ray.get(res)

In [ ]:
res = [sum(el) for el in list(zip(*res))]

In [ ]:
print('count: ', res[0])
print('size: ', res[1] / 1000000000000, 'Tb')

## Dedup repo count and names by stars

In [ ]:
paths = list(Path('/data/hf_repos/the_stack_v1_1_near_dedup_parquet/data/java/').glob('*.parquet'))

In [ ]:
df = pd.read_parquet(paths[0])

In [ ]:
df

In [ ]:
len(set(df['max_stars_repo_name']))

In [ ]:
@ray.remote(scheduling_strategy="SPREAD")
def get_java_repos_stats(bucket_filename):
    df = pd.read_parquet(bucket_filename)
    return set(df['max_stars_repo_name'])
    

In [ ]:
res = []
for path in paths:
    res += [get_java_repos_stats.remote(path)]

In [ ]:
res = ray.get(res)

In [ ]:
repos = set().union(*res)

In [ ]:
len(repos)

## Size of those repos

In [ ]:
paths = list(Path('/data/hf_repos/the-stack-metadata').glob('data/*'))

In [ ]:
df_ri = pd.read_parquet(
    paths[0] / 'ri.parquet'
)
ri_ids = set(df_ri[df_ri['name'].isin(repos)]['id'])
df_fi = pd.read_parquet(
    paths[0] / 'fi.parquet'
)
df_fi = df_fi[(df_fi['ri_id'].isin(ri_ids)) & (df_fi['lang_ex'] == 'Java') & (df_fi['is_deleted'] == False)]

In [ ]:
df_fi

In [ ]:
@ray.remote(scheduling_strategy="SPREAD")
def get_java_repos_stats(bucket_path, repos):
    df_ri = pd.read_parquet(
        bucket_path / 'ri.parquet'
    )
    ri_ids = set(df_ri[df_ri['name'].isin(repos)]['id'])
    df_fi = pd.read_parquet(
        bucket_path / 'fi.parquet'
    )
    df_fi = df_fi[(df_fi['ri_id'].isin(ri_ids)) & (df_fi['lang_ex'] == 'Java') & (df_fi['is_deleted'] == False)]
    return df_fi['ri_id'].nunique(), df_fi['size'].sum(), len(df_fi)

In [ ]:
repos_ref = ray.put(repos)

In [ ]:
res = []
for path in paths:
    res += [get_java_repos_stats.remote(path, repos_ref)]

In [ ]:
res = ray.get(res)

In [ ]:
res = [sum(el) for el in list(zip(*res))]

In [ ]:
print('repos count: ', res[0])
print('size: ', res[1] / 1000000000000, 'Tb')
print('files count: ', res[2])

In [ ]:
server.scale_cluster(0)

## Assign and save repository to bucket mapping info

In [ ]:
len(repos)

In [ ]:
usernames = set(el.split('/')[0] for el in repos)

In [ ]:
len(usernames)

In [ ]:
repos = list(repos)

In [ ]:
repos.sort()

In [ ]:
repo_2_bucket = {}
for i, el in enumerate(repos):
        repo_2_bucket[el] = i // 1000

In [ ]:
repo_2_bucket[repos[-1]]

In [ ]:
with open('/data/hf_repos/the-stak-repo-level/meta_data/java/repo_to_bucket.json', 'wt') as f:
    json.dump(repo_2_bucket, f)